In [11]:
import os
import re
import numpy as np
import tifffile as tiff
import cv2
import math

In [12]:
def parse_string(input_string):
    pattern = r"R\d{2}-C\d{2}-F\d+_(\d{5})_(IM|CM)_(KIF11_orig|LUC_orig)"
    match = re.search(pattern, input_string)
    if match:
        number, mode, gene = match.groups()
        return number, mode, gene
    else:
        raise ValueError("Input string does not match the expected format")

def max_projection_montage(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok = True)
    grouped_files = {}

    # Group files by number and mode
    for filename in os.listdir(input_folder):
        if filename.endswith(".tif"):
            try:
                number, mode, gene = parse_string(filename)
                key = (number, mode)
                if key not in grouped_files:
                    grouped_files[key] = {"KIF11_orig": [], "LUC_orig": []}
                grouped_files[key][gene].append(filename)
            except ValueError:
                continue

    # Process each group
    for (number, mode), genes in grouped_files.items():
        projections = {"KIF11_orig": [], "LUC_orig": []}
        
        max_projections = 40  # Set the limit

        for gene, file_list in genes.items():
            file_list.sort()

            # Limit the number of files
          #  if len(file_list) > max_projections:
          #      file_list = file_list[:max_projections]  # First N images
                
            for filename in file_list:
                input_path = os.path.join(input_folder, filename)
                with tiff.TiffFile(input_path) as tif_file:
                    img = tif_file.asarray()
                
                if len(img.shape) == 4 and img.shape[1] == 4:
                    img = np.moveaxis(img, 1, 0)
                    img = img[:, 5:16, :, :]
                    ch3 = np.max(img[1], axis=0)
                    ch2 = np.max(img[3], axis=0)

                    # Find the brightest object in channel 3
                    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(ch3)
                    center_x, center_y = max_loc
                    
                    if max_val == 0:
                        continue
                    
                    # Define cropping bounds (250x250 pixels around the brightest object)
                    half_size = 125
                    h, w = ch3.shape
                    x1, x2 = center_x - half_size, center_x + half_size
                    y1, y2 = center_y - half_size, center_y + half_size

                    pad_x1, pad_x2, pad_y1, pad_y2 = 0, 0, 0, 0
                    if x1 < 0:
                        pad_x1 = abs(x1)
                        x1 = 0
                    if x2 > w:
                        pad_x2 = x2 - w
                        x2 = w
                    if y1 < 0:
                        pad_y1 = abs(y1)
                        y1 = 0
                    if y2 > h:
                        pad_y2 = y2 - h
                        y2 = h
                    
                    ch3_crop = ch3[y1:y2, x1:x2]
                    ch2_crop = ch2[y1:y2, x1:x2]
                    
                    ch3_padded = cv2.copyMakeBorder(ch3_crop, pad_y1, pad_y2, pad_x1, pad_x2, cv2.BORDER_CONSTANT, value=0)
                    ch2_padded = cv2.copyMakeBorder(ch2_crop, pad_y1, pad_y2, pad_x1, pad_x2, cv2.BORDER_CONSTANT, value=0)
                    

                    ch3_gray = cv2.normalize(ch3_padded, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
                    ch2_gray = cv2.normalize(ch2_padded, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
                    merged = cv2.merge((ch3_gray, ch3_gray, np.zeros_like(ch3_gray)))
                    grayscale = cv2.merge((ch2_gray, ch2_gray, ch2_gray))
                    final_merged = np.maximum(merged, grayscale)

                    projections[gene].append(final_merged)

        # Create montage
        if projections["KIF11_orig"] or projections["LUC_orig"]:
            max_images = max(len(projections["KIF11_orig"]), len(projections["LUC_orig"]))
            rows = math.ceil(max_images / 5)  # Dynamically determine rows
            img_h, img_w, _ = projections["KIF11_orig"][0].shape if projections["KIF11_orig"] else projections["LUC_orig"][0].shape
            montage = np.zeros((img_h * rows, img_w * 10, 3), dtype=np.uint8)  # 10 columns total

            for idx, img in enumerate(projections["KIF11_orig"]):
                row, col = divmod(idx, 5)
                y1, y2 = row * img_h, (row + 1) * img_h
                x1, x2 = col * img_w, (col + 1) * img_w
                montage[y1:y2, x1:x2] = img

            for idx, img in enumerate(projections["LUC_orig"]):
                row, col = divmod(idx, 5)
                y1, y2 = row * img_h, (row + 1) * img_h
                x1, x2 = (col + 5) * img_w, (col + 6) * img_w  # Shift for LUC_init
                montage[y1:y2, x1:x2] = img
                    

            output_path = os.path.join(output_folder, f"Montage_{number}_{mode}.tif")
            with tiff.TiffWriter(output_path) as tif:
                tif.write(montage)
            print(f"Created montage: {output_path}")

In [13]:
input_folder = "/Volumes/arxivBeta/_Tobias/Opera/20250314/01_Importer"  
output_folder = "/Volumes/arxivBeta/_Tobias/Opera/20250314/02_Galleries_V1" 

max_projection_montage(input_folder, output_folder)

Created montage: /Volumes/arxivBeta/_Tobias/Opera/20250314/02_Galleries_V1/Montage_20000_IM.tif
Created montage: /Volumes/arxivBeta/_Tobias/Opera/20250314/02_Galleries_V1/Montage_60000_CM.tif
Created montage: /Volumes/arxivBeta/_Tobias/Opera/20250314/02_Galleries_V1/Montage_50000_CM.tif
Created montage: /Volumes/arxivBeta/_Tobias/Opera/20250314/02_Galleries_V1/Montage_40000_IM.tif
Created montage: /Volumes/arxivBeta/_Tobias/Opera/20250314/02_Galleries_V1/Montage_30000_IM.tif
Created montage: /Volumes/arxivBeta/_Tobias/Opera/20250314/02_Galleries_V1/Montage_60000_IM.tif
Created montage: /Volumes/arxivBeta/_Tobias/Opera/20250314/02_Galleries_V1/Montage_50000_IM.tif
Created montage: /Volumes/arxivBeta/_Tobias/Opera/20250314/02_Galleries_V1/Montage_40000_CM.tif
Created montage: /Volumes/arxivBeta/_Tobias/Opera/20250314/02_Galleries_V1/Montage_30000_CM.tif
Created montage: /Volumes/arxivBeta/_Tobias/Opera/20250314/02_Galleries_V1/Montage_20000_CM.tif
